In [1]:
#pip install googletrans==4.0.0-rc1
# pip install -U easynmt

In [2]:
# pip install fasttext
# pip install -U easynmt

In [3]:
import numpy as np
import time 
import pandas as pd 
import matplotlib.pyplot as plt
import googletrans
from googletrans import Translator
import tqdm
from easynmt import EasyNMT

# Extraction of data 

In [4]:
content = pd.read_csv(r'C:\Users\mehdy\Desktop\Work\KAGGLE\learning-equality-curriculum-recommendations\data\content.csv', sep=',')
correlation = pd.read_csv(r'C:\Users\mehdy\Desktop\Work\KAGGLE\learning-equality-curriculum-recommendations\data\correlations.csv', sep=',')
topics = pd.read_csv(r'C:\Users\mehdy\Desktop\Work\KAGGLE\learning-equality-curriculum-recommendations\data\topics.csv', sep=',')
sample_subm = pd.read_csv(r'C:\Users\mehdy\Desktop\Work\KAGGLE\learning-equality-curriculum-recommendations\data\sample_submission.csv', sep= ',')

In [5]:
print('data content shape :', content.shape)
print('data correlation shape :', correlation.shape)
print('data topics shape :', topics.shape)
print('data sample sub shape :', sample_subm.shape)

data content shape : (154047, 8)
data correlation shape : (61517, 2)
data topics shape : (76972, 9)
data sample sub shape : (5, 2)


In [6]:
(correlation.topic_id)

0        t_00004da3a1b2
1        t_00068291e9a4
2        t_00069b63a70a
3        t_0006d41a73a8
4        t_0008768bdee6
              ...      
61512    t_fff830472691
61513    t_fff9e5407d13
61514    t_fffbe1d5d43c
61515    t_fffe14f1be1e
61516    t_fffe811a6da9
Name: topic_id, Length: 61517, dtype: object

In [7]:
sample_subm

,topic_id,content_ids
0,t_00004da3a1b2,c_1108dd0c7a5d c_376c5a8eb028 c_5bc0e1e2cba0 c...
1,t_00068291e9a4,c_639ea2ef9c95 c_89ce9367be10 c_ac1672cdcd2c c...
2,t_00069b63a70a,c_11a1dc0bfb99
3,t_0006d41a73a8,c_0c6473c3480d c_1c57a1316568 c_5e375cf14c47 c...
4,t_4054df11a74e,c_3695c5dc1df6 c_f2d184a98231


In [8]:
correlation

,topic_id,content_ids
0,t_00004da3a1b2,c_1108dd0c7a5d c_376c5a8eb028 c_5bc0e1e2cba0 c...
1,t_00068291e9a4,c_639ea2ef9c95 c_89ce9367be10 c_ac1672cdcd2c c...
2,t_00069b63a70a,c_11a1dc0bfb99
3,t_0006d41a73a8,c_0c6473c3480d c_1c57a1316568 c_5e375cf14c47 c...
4,t_0008768bdee6,c_34e1424229b4 c_7d1a964d66d5 c_aab93ee667f4
...,...,...
61512,t_fff830472691,c_61fb63326e5d c_8f224e321c87
61513,t_fff9e5407d13,c_026db653a269 c_0fb048a6412c c_20de77522603 c...
61514,t_fffbe1d5d43c,c_46f852a49c08 c_6659207b25d5
61515,t_fffe14f1be1e,c_cece166bad6a


In [9]:
content.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154047 entries, 0 to 154046
Data columns (total 8 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   id                154047 non-null  object
 1   title             154038 non-null  object
 2   description       89456 non-null   object
 3   kind              154047 non-null  object
 4   text              74035 non-null   object
 5   language          154047 non-null  object
 6   copyright_holder  71821 non-null   object
 7   license           74035 non-null   object
dtypes: object(8)
memory usage: 9.4+ MB


In [10]:
correlation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61517 entries, 0 to 61516
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   topic_id     61517 non-null  object
 1   content_ids  61517 non-null  object
dtypes: object(2)
memory usage: 961.3+ KB


In [11]:
topics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76972 entries, 0 to 76971
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           76972 non-null  object
 1   title        76970 non-null  object
 2   description  34953 non-null  object
 3   channel      76972 non-null  object
 4   category     76972 non-null  object
 5   level        76972 non-null  int64 
 6   language     76972 non-null  object
 7   parent       76801 non-null  object
 8   has_content  76972 non-null  bool  
dtypes: bool(1), int64(1), object(7)
memory usage: 4.8+ MB


# Create data train and test with validation by merging topics and contents id

In [12]:
correlation.head()

,topic_id,content_ids
0,t_00004da3a1b2,c_1108dd0c7a5d c_376c5a8eb028 c_5bc0e1e2cba0 c...
1,t_00068291e9a4,c_639ea2ef9c95 c_89ce9367be10 c_ac1672cdcd2c c...
2,t_00069b63a70a,c_11a1dc0bfb99
3,t_0006d41a73a8,c_0c6473c3480d c_1c57a1316568 c_5e375cf14c47 c...
4,t_0008768bdee6,c_34e1424229b4 c_7d1a964d66d5 c_aab93ee667f4


In [13]:
correlation_bis = correlation[:]
correlation_bis['content_ids'] = correlation_bis.content_ids.apply(lambda x: x.split(' '))
correlation_bis = correlation_bis.explode("content_ids")
correlation_bis

,topic_id,content_ids
0,t_00004da3a1b2,c_1108dd0c7a5d
0,t_00004da3a1b2,c_376c5a8eb028
0,t_00004da3a1b2,c_5bc0e1e2cba0
0,t_00004da3a1b2,c_76231f9d0b5e
1,t_00068291e9a4,c_639ea2ef9c95
...,...,...
61513,t_fff9e5407d13,c_d64037a72376
61514,t_fffbe1d5d43c,c_46f852a49c08
61514,t_fffbe1d5d43c,c_6659207b25d5
61515,t_fffe14f1be1e,c_cece166bad6a


In [14]:
correlation_bis[correlation_bis.topic_id == 't_00004da3a1b2']

,topic_id,content_ids
0,t_00004da3a1b2,c_1108dd0c7a5d
0,t_00004da3a1b2,c_376c5a8eb028
0,t_00004da3a1b2,c_5bc0e1e2cba0
0,t_00004da3a1b2,c_76231f9d0b5e


In [15]:
correlation_bis[correlation_bis.content_ids == 'c_1108dd0c7a5d']

,topic_id,content_ids
0,t_00004da3a1b2,c_1108dd0c7a5d
15070,t_3f5ce16d83ff,c_1108dd0c7a5d


All topics don't have contents id but all contents id have topicvs ids

In [16]:
# Train and test separation
topic = topics.id.tolist()
conten = content.id.tolist()

train_topic = correlation_bis.topic_id.unique().tolist()
train_content = correlation_bis.content_ids.unique().tolist()
len(train_topic)
len(train_content)

154047

In [17]:
train_top = topics[topics.id.isin(train_topic)]
test_top = topics[-topics.id.isin(train_topic)]
train_top.shape

(61517, 9)

# Translation in english first 

In [18]:
# dictionnary with name of language 
iso_639_choices = [('ab', 'Abkhaz'),
('aa', 'Afar'),
('af', 'Afrikaans'),
('ak', 'Akan'),
('sq', 'Albanian'),
('am', 'Amharic'),
('ar', 'Arabic'),
('an', 'Aragonese'),
('hy', 'Armenian'),
('as', 'Assamese'),
('av', 'Avaric'),
('ae', 'Avestan'),
('ay', 'Aymara'),
('az', 'Azerbaijani'),
('bm', 'Bambara'),
('ba', 'Bashkir'),
('eu', 'Basque'),
('be', 'Belarusian'),
('bn', 'Bengali'),
('bh', 'Bihari'),
('bi', 'Bislama'),
('bs', 'Bosnian'),
('br', 'Breton'),
('bg', 'Bulgarian'),
('my', 'Burmese'),
('ca', 'Catalan; Valencian'),
('ch', 'Chamorro'),
('ce', 'Chechen'),
('ny', 'Chichewa; Chewa; Nyanja'),
('zh', 'Chinese'),
('cv', 'Chuvash'),
('kw', 'Cornish'),
('co', 'Corsican'),
('cr', 'Cree'),
('hr', 'Croatian'),
('cs', 'Czech'),
('da', 'Danish'),
('dv', 'Divehi; Maldivian;'),
('nl', 'Dutch'),
('dz', 'Dzongkha'),
('en', 'English'),
('eo', 'Esperanto'),
('et', 'Estonian'),
('ee', 'Ewe'),
('fo', 'Faroese'),
('fj', 'Fijian'),
('fi', 'Finnish'),
('fr', 'French'),
('ff', 'Fula'),
('gl', 'Galician'),
('ka', 'Georgian'),
('de', 'German'),
('el', 'Greek, Modern'),
('gn', 'Guaraní'),
('gu', 'Gujarati'),
('ht', 'Haitian'),
('ha', 'Hausa'),
('he', 'Hebrew (modern)'),
('hz', 'Herero'),
('hi', 'Hindi'),
('ho', 'Hiri Motu'),
('hu', 'Hungarian'),
('ia', 'Interlingua'),
('id', 'Indonesian'),
('ie', 'Interlingue'),
('ga', 'Irish'),
('ig', 'Igbo'),
('ik', 'Inupiaq'),
('io', 'Ido'),
('is', 'Icelandic'),
('it', 'Italian'),
('iu', 'Inuktitut'),
('ja', 'Japanese'),
('jv', 'Javanese'),
('kl', 'Kalaallisut'),
('kn', 'Kannada'),
('kr', 'Kanuri'),
('ks', 'Kashmiri'),
('kk', 'Kazakh'),
('km', 'Khmer'),
('ki', 'Kikuyu, Gikuyu'),
('rw', 'Kinyarwanda'),
('ky', 'Kirghiz, Kyrgyz'),
('kv', 'Komi'),
('kg', 'Kongo'),
('ko', 'Korean'),
('ku', 'Kurdish'),
('kj', 'Kwanyama, Kuanyama'),
('la', 'Latin'),
('lb', 'Luxembourgish'),
('lg', 'Luganda'),
('li', 'Limburgish'),
('ln', 'Lingala'),
('lo', 'Lao'),
('lt', 'Lithuanian'),
('lu', 'Luba-Katanga'),
('lv', 'Latvian'),
('gv', 'Manx'),
('mk', 'Macedonian'),
('mg', 'Malagasy'),
('ms', 'Malay'),
('ml', 'Malayalam'),
('mt', 'Maltese'),
('mi', 'Māori'),
('mr', 'Marathi (Marāṭhī)'),
('mh', 'Marshallese'),
('mn', 'Mongolian'),
('na', 'Nauru'),
('nv', 'Navajo, Navaho'),
('nb', 'Norwegian Bokmål'),
('nd', 'North Ndebele'),
('ne', 'Nepali'),
('ng', 'Ndonga'),
('nn', 'Norwegian Nynorsk'),
('no', 'Norwegian'),
('ii', 'Nuosu'),
('nr', 'South Ndebele'),
('oc', 'Occitan'),
('oj', 'Ojibwe, Ojibwa'),
('cu', 'Old Church Slavonic'),
('om', 'Oromo'),
('or', 'Oriya'),
('os', 'Ossetian, Ossetic'),
('pa', 'Panjabi, Punjabi'),
('pi', 'Pāli'),
('fa', 'Persian'),
('pl', 'Polish'),
('ps', 'Pashto, Pushto'),
('pt', 'Portuguese'),
('qu', 'Quechua'),
('rm', 'Romansh'),
('rn', 'Kirundi'),
('ro', 'Romanian, Moldavan'),
('ru', 'Russian'),
('sa', 'Sanskrit (Saṁskṛta)'),
('sc', 'Sardinian'),
('sd', 'Sindhi'),
('se', 'Northern Sami'),
('sm', 'Samoan'),
('sg', 'Sango'),
('sr', 'Serbian'),
('gd', 'Scottish Gaelic'),
('sn', 'Shona'),
('si', 'Sinhala, Sinhalese'),
('sk', 'Slovak'),
('sl', 'Slovene'),
('so', 'Somali'),
('st', 'Southern Sotho'),
('es', 'Spanish; Castilian'),
('su', 'Sundanese'),
('sw', 'Swahili'),
('ss', 'Swati'),
('sv', 'Swedish'),
('ta', 'Tamil'),
('te', 'Telugu'),
('tg', 'Tajik'),
('th', 'Thai'),
('ti', 'Tigrinya'),
('bo', 'Tibetan'),
('tk', 'Turkmen'),
('tl', 'Tagalog'),
('tn', 'Tswana'),
('to', 'Tonga'),
('tr', 'Turkish'),
('ts', 'Tsonga'),
('tt', 'Tatar'),
('tw', 'Twi'),
('ty', 'Tahitian'),
('ug', 'Uighur, Uyghur'),
('uk', 'Ukrainian'),
('ur', 'Urdu'),
('uz', 'Uzbek'),
('ve', 'Venda'),
('vi', 'Vietnamese'),
('vo', 'Volapük'),
('wa', 'Walloon'),
('cy', 'Welsh'),
('wo', 'Wolof'),
('fy', 'Western Frisian'),
('xh', 'Xhosa'),
('yi', 'Yiddish'),
('yo', 'Yoruba'),
('za', 'Zhuang, Chuang'),
('zu', 'Zulu'),]
iso_639_choices = pd.DataFrame(iso_639_choices)
iso_639_choices.columns = ['Abrev', 'Long_language']
iso_639_choices

,Abrev,Long_language
0,ab,Abkhaz
1,aa,Afar
2,af,Afrikaans
3,ak,Akan
4,sq,Albanian
...,...,...
179,xh,Xhosa
180,yi,Yiddish
181,yo,Yoruba
182,za,"Zhuang, Chuang"


In [19]:
content_copy = content[:]
content_copy['title'] = content_copy['title'] .astype('str')

In [20]:
content_copy.head()

,id,title,description,kind,text,language,copyright_holder,license
0,c_00002381196d,"Sumar números de varios dígitos: 48,029+233,930","Suma 48,029+233,930 mediante el algoritmo está...",video,NaN,es,NaN,NaN
1,c_000087304a9e,Trovare i fattori di un numero,Sal trova i fattori di 120.\n\n,video,NaN,it,NaN,NaN
2,c_0000ad142ddb,Sumar curvas de demanda,Cómo añadir curvas de demanda\n\n,video,NaN,es,NaN,NaN
3,c_0000c03adc8d,Nado de aproximação,Neste vídeo você vai aprender o nado de aproxi...,document,\nNado de aproximação\nSaber nadar nas ondas ...,pt,Sikana Education,CC BY-NC-ND
4,c_00016694ea2a,geometry-m3-topic-a-overview.pdf,geometry-m3-topic-a-overview.pdf,document,Estándares Comunes del Estado de Nueva York\n\...,es,Engage NY,CC BY-NC-SA


In [24]:
toto = content_copy.iloc[:10]
toto

,id,title,description,kind,text,language,copyright_holder,license
0,c_00002381196d,"Sumar números de varios dígitos: 48,029+233,930","Suma 48,029+233,930 mediante el algoritmo está...",video,NaN,es,NaN,NaN
1,c_000087304a9e,Trovare i fattori di un numero,Sal trova i fattori di 120.\n\n,video,NaN,it,NaN,NaN
2,c_0000ad142ddb,Sumar curvas de demanda,Cómo añadir curvas de demanda\n\n,video,NaN,es,NaN,NaN
3,c_0000c03adc8d,Nado de aproximação,Neste vídeo você vai aprender o nado de aproxi...,document,\nNado de aproximação\nSaber nadar nas ondas ...,pt,Sikana Education,CC BY-NC-ND
4,c_00016694ea2a,geometry-m3-topic-a-overview.pdf,geometry-m3-topic-a-overview.pdf,document,Estándares Comunes del Estado de Nueva York\n\...,es,Engage NY,CC BY-NC-SA
5,c_00019840d110,5.12E: Regulation of the Calvin Cycle,NaN,html5,LEARNING OBJECTIVES\n\nOutline the three major...,en,CSU and Merlot,CC BY-NC-SA
6,c_0001ec56e20f,Reflexionemos sobre lo que vemos y escuchamos,NaN,document,Lección\n\n7\n\nReflexionemos sobre lo que\nve...,es,Publicado por el Lic. Edelberto Andino(edelber...,CC BY-NC-SA
7,c_00025aaa1533,अंग्रेजी ओके प्लीज 1.2,source_url=http://www.prathamopenschool.org/Co...,video,NaN,mr,NaN,NaN
8,c_00027d03ca7d,4.E: Genomes and Chromosomes (Exercises),NaN,html5,4.3\n\n(BPA) Answer the following questions wi...,en,CSU and Merlot,CC BY-NC-SA
9,c_000314eb850f,La banca 12: los bonos del tesoro (deuda pública),Introducción a la deuda y los fondos públicos....,video,NaN,es,NaN,NaN


In [31]:
import tqdm
translator = EasyNMT('opus-mt')
translations = {}
for columns in ['title']:
    print(columns)
    for element in toto[columns]:
        translations[element] =  translator.translate(element, target_lang='en')
translations

title


c:\Users\mehdy\anaconda3\lib\site-packages\transformers\models\marian\tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
c:\Users\mehdy\anaconda3\lib\site-packages\transformers\generation\utils.py:1387: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 512 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Exception: Helsinki-NLP/opus-mt-pt-en is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo with `use_auth_token` or log in with `huggingface-cli login` and pass `use_auth_token=True`.


OSError: Helsinki-NLP/opus-mt-pt-en is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo with `use_auth_token` or log in with `huggingface-cli login` and pass `use_auth_token=True`.

In [21]:
import tqdm
translator = EasyNMT('opus-mt')
translations = {}
for columns in ['title', 'description', 'text']:
    print(columns)
    unique_element = content_copy[columns].unique()
    for element in unique_element:
        # add dictionnary 
        translations[element] = translator.translate(element, target_lang='en').text
translations 

In [35]:
translator = Translator()
translations = {}
a = 0
for columns in ['title', 'description', 'text']:
    print(columns)
    for element in toto[columns]:
        print(element)
        translations[str(a)] = translator.translate(element, dest='en').text
        a = a + 1
    #unique_element = toto[columns].unique()
    #for element in unique_element:
    #    time.sleep(0.3)
    #    # add dictionnary 
    #    translations[element] = translator.translate(element, dest='en').text
translations 

title
Sumar números de varios dígitos: 48,029+233,930 
Trovare i fattori di un numero
Sumar curvas de demanda
Nado de aproximação
geometry-m3-topic-a-overview.pdf
5.12E: Regulation of the Calvin Cycle
Reflexionemos sobre lo que vemos y escuchamos
अंग्रेजी ओके प्लीज 1.2
4.E: Genomes and Chromosomes (Exercises)
La banca 12: los bonos del tesoro (deuda pública)
description
Suma 48,029+233,930 mediante el algoritmo estándar.


Sal trova i fattori di 120.


Cómo añadir curvas de demanda


Neste vídeo você vai aprender o nado de aproximação, técnica para manter a observação na zona de arrebentação do surf.
geometry-m3-topic-a-overview.pdf
nan
nan
source_url=http://www.prathamopenschool.org/CourseContent/videos/AOK_1.2.mp4
nan
Introducción a la deuda y los fondos públicos. Qué significa cuando decimos
que los pagarés de la Reserva Federal son emitidos por el banco de la reserva,
pero son una obligación del gobierno.


text
nan
nan
nan

Nado de aproximação
Saber nadar nas ondas sem a prancha 

TypeError: the JSON object must be str, bytes or bytearray, not NoneType

In [33]:
translations

{'Sumar números de varios dígitos: 48,029+233,930 ': 'Add several digit numbers: 48,029+233,930',
 'Trovare i fattori di un numero': 'Find the factors of a number',
 'Sumar curvas de demanda': 'Add demand curves',
 'Nado de aproximação': 'Approach',
 'geometry-m3-topic-a-overview.pdf': 'geometry-m3-topic-a-overview.pdf',
 '5.12E: Regulation of the Calvin Cycle': '5.12E: Regulation of the Calvin Cycle',
 'Reflexionemos sobre lo que vemos y escuchamos': "Let's reflect on what we see and listen",
 'अंग्रेजी ओके प्लीज 1.2': 'English Ok please 1.2',
 '4.E: Genomes and Chromosomes (Exercises)': '4.E: Genomes and Chromosomes (Exercises)',
 'La banca 12: los bonos del tesoro (deuda pública)': 'Banking 12: Treasury Bonds (Public Debt)',
 '9.3: Van Der Waals Forces between Atoms': '9.3: Van der Waals Forces Between Atoms',
 'Compare multi-digit numbers': 'Compare multi-digit numbers',
 'Ендомембранна система': 'Endomembrane',
 'સમક્ષિતિજ પ્રક્ષિપ્ત માટે શુદ્ધ ગતિવિજ્ઞાનના સમીકરણને ઉકેલવું': 'Res

In [30]:
translator = Translator()
translations = {}
for columns in ['title', 'description', 'text']:
    print(columns)
    content_copy[columns] = content_copy[columns].apply(lambda x: translator.translate(x, dest='en').text)

title


AttributeError: 'Translator' object has no attribute 'raise_Exception'

In [ ]:
topic_copy = topics[:]

In [87]:
topics[topics.id == correlation.topic_id[0]]

,id,title,description,channel,category,level,language,parent,has_content
0,t_00004da3a1b2,Откриването на резисторите,"Изследване на материали, които предизвикват на...",000cf7,source,4,bg,t_16e29365b50d,True


In [97]:
correlation.loc[0][1]

'c_1108dd0c7a5d c_376c5a8eb028 c_5bc0e1e2cba0 c_76231f9d0b5e'

In [88]:
content[content['id'].isin(correlation.content_ids.iloc[0].split(' '))]

,id,title,description,kind,text,language,copyright_holder,license
10216,c_1108dd0c7a5d,Молив като резистор,"Моливът причинява промяна в отклонението, подо...",video,NaN,bg,NaN,NaN
33412,c_376c5a8eb028,Да чуем променливото съпротивление,Тук чертаем линия на лист хартия и я използвам...,video,NaN,bg,NaN,NaN
55283,c_5bc0e1e2cba0,Променлив резистор (реостат) с графит от молив,Използваме сърцевината на молива (неговия граф...,video,NaN,bg,NaN,NaN
70957,c_76231f9d0b5e,Последователно свързване на галваничен елемент...,"Защо отклонението се променя, когато се свърже...",video,NaN,bg,NaN,NaN
